In [1]:
import collections
import requests
import json
import re
import os
import sys

with open('../../bundestagsreden parser/speeches_20.jsonl', 'r', encoding='utf8') as fp:
    data = list(fp)
speeches = []
for line in data:
    speeches.append(json.loads(line))

In [2]:
# globale items
regex = "{(.*?)}" # hier die genaue Definition der Kommentare vom Parser <<< >>>
kommentare = []

In [3]:
# Funktion: speeches_comments
def extract_comments(speeches):
    comments = []
    modified_speeches = []
    for speech in speeches:
        text = speech['text']
        matches = re.findall(regex,text)
        speech['comments'] = matches
        modified_speeches.append(speech)
    return modified_speeches
# Funktionsaufruf:  speeches_comments = extract_comments(speeches)
speeches_comments = extract_comments(speeches)
# Funktionsausgabe: print(speeches_comments[0]['comments'])
#                   print(speeches_comments[0]['party'])

In [4]:
# Funktion: specific_and_non_search
def filter_with_complex_non_words(search_terms,non_words,speeches):
    filtered_speeches = []
    for speech in speeches:
        for comment in speech['comments']:
            if all([term in comment for term in search_terms]) and not any([term in comment for term in non_words]):
                filtered_speeches.append(speech)
                break
    return filtered_speeches
# Funktionsaufruf:  specific_and_non_search = filter_with_complex_non_words(['Beifall','CDU'],['Zuruf'],speeches_comments)
# Funktionsausgabe: print(specific_and_non_search[0])                               enthält AND enthält nicht
#                   print(len(specific_and_non_search))

In [5]:
specific_and_non_search = filter_with_complex_non_words(['Zuruf','CDU'],['Beifall'],speeches_comments)
print(f'Zuruf für CDU: {len(specific_and_non_search)}')

Zuruf für CDU: 233


In [6]:
specific_and_non_search = filter_with_complex_non_words(['Zuruf','SPD'],['Beifall'],speeches_comments)
print(f'Zuruf für SPD:           {len(specific_and_non_search)}')
specific_and_non_search = filter_with_complex_non_words(['Zuruf','Beifall','SPD'],[],speeches_comments)
print(f'Beifall & Zuruf für SPD: {len(specific_and_non_search)}')

Zuruf für SPD:           244
Beifall & Zuruf für SPD: 333


In [7]:
Beifall_and_CDU   = ['Beifall','CDU']
Beifall_and_SPD   = ['Beifall','SPD']
Beifall_and_Linke = ['Beifall','LINKE']
Beifall_and_GRÜNE = ['Beifall','GRÜNE']
Beifall_and_FDP   = ['Beifall','FDP']
Beifall_and_AfD   = ['Beifall','AfD']
and_not_Zuruf     = ['Zuruf']
mega_list_beifall = [Beifall_and_CDU,Beifall_and_SPD,Beifall_and_Linke,Beifall_and_GRÜNE,Beifall_and_AfD,Beifall_and_FDP]
Zuruf_and_CDU     = ['Zuruf','CDU']
Zuruf_and_SPD     = ['Zuruf','SPD']
Zuruf_and_Linke   = ['Zuruf','LINKE']
Zuruf_and_GRÜNE   = ['Zuruf','GRÜNE']
Zuruf_and_FDP     = ['Zuruf','FDP']
Zuruf_and_AfD     = ['Zuruf','AfD']
and_not_beifall   = ['Beifall']
mega_list_zuruf   = [Zuruf_and_CDU,Zuruf_and_SPD,Zuruf_and_Linke,Zuruf_and_GRÜNE,Zuruf_and_FDP,Zuruf_and_AfD]
both_CDU          = ['Beifall','Zuruf','CDU']
both_SPD          = ['Beifall','Zuruf','SPD']
both_Linke        = ['Beifall','Zuruf','LINKE']
both_GRÜNE        = ['Beifall','Zuruf','GRÜNE']
both_FDP          = ['Beifall','Zuruf','FDP']
both_AfD          = ['Beifall','Zuruf','AfD']
and_not_nothing   = []
mega_list_both    = [both_CDU,both_SPD,both_Linke,both_GRÜNE,both_FDP,both_AfD]

In [8]:
result_list_beifall = []
for subList_beifall in mega_list_beifall:
    result_list_beifall.append(filter_with_complex_non_words(subList_beifall,and_not_Zuruf,speeches_comments))
result_list_zuruf = []
for subList_zuruf in mega_list_zuruf:
    result_list_zuruf.append  (filter_with_complex_non_words(subList_zuruf,and_not_beifall,speeches_comments))
result_list_both = []
for subList_both in mega_list_both:
    result_list_both.append   (filter_with_complex_non_words(subList_both,and_not_nothing,speeches_comments))

In [12]:
for (results_beifall,subList_beifall) in zip (result_list_beifall,mega_list_beifall):
    print(f'Anzahl Kommentaren \'{subList_beifall[0]}\' für \'{subList_beifall[1]}\' UND ohne \'{and_not_Zuruf[0]}\' ist: {len(results_beifall)}')
for (results_zuruf,subList_zuruf) in zip (result_list_zuruf,mega_list_zuruf):
    print(f'Anzahl Kommentaren \'{subList_zuruf[0]}\' für \'{subList_zuruf[1]}\' UND ohne \'{and_not_beifall[0]}\' ist: {len(results_zuruf)}')
for (results_both,subList_both) in zip (result_list_both,mega_list_both):
    print(f'Anzahl Kommentaren mit \'{subList_both[0]}\' UND \'{subList_both[1]}\' für \'{subList_both[2]}\' ist: {len(results_both)}')    

Anzahl Kommentaren 'Beifall' für 'CDU' UND ohne 'Zuruf' ist: 1497
Anzahl Kommentaren 'Beifall' für 'SPD' UND ohne 'Zuruf' ist: 2237
Anzahl Kommentaren 'Beifall' für 'LINKE' UND ohne 'Zuruf' ist: 905
Anzahl Kommentaren 'Beifall' für 'GRÜNE' UND ohne 'Zuruf' ist: 2190
Anzahl Kommentaren 'Beifall' für 'AfD' UND ohne 'Zuruf' ist: 940
Anzahl Kommentaren 'Beifall' für 'FDP' UND ohne 'Zuruf' ist: 2145
Anzahl Kommentaren 'Zuruf' für 'CDU' UND ohne 'Beifall' ist: 233
Anzahl Kommentaren 'Zuruf' für 'SPD' UND ohne 'Beifall' ist: 244
Anzahl Kommentaren 'Zuruf' für 'LINKE' UND ohne 'Beifall' ist: 86
Anzahl Kommentaren 'Zuruf' für 'GRÜNE' UND ohne 'Beifall' ist: 191
Anzahl Kommentaren 'Zuruf' für 'FDP' UND ohne 'Beifall' ist: 133
Anzahl Kommentaren 'Zuruf' für 'AfD' UND ohne 'Beifall' ist: 335
Anzahl Kommentaren mit 'Beifall' UND 'Zuruf' für 'CDU' ist: 285
Anzahl Kommentaren mit 'Beifall' UND 'Zuruf' für 'SPD' ist: 333
Anzahl Kommentaren mit 'Beifall' UND 'Zuruf' für 'LINKE' ist: 101
Anzahl Kommenta